# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f22111721f0>
├── 'a' --> tensor([[ 1.0839, -0.7698, -0.9704],
│                   [ 0.4433, -0.7054,  1.1089]])
└── 'x' --> <FastTreeValue 0x7f22111c9af0>
    └── 'c' --> tensor([[-1.8767,  0.5016, -0.4694,  0.3864],
                        [-0.7104, -0.9451,  0.5307,  0.4539],
                        [-0.1516, -0.4768, -0.9145,  0.4315]])

In [4]:
t.a

tensor([[ 1.0839, -0.7698, -0.9704],
        [ 0.4433, -0.7054,  1.1089]])

In [5]:
%timeit t.a

63.2 ns ± 0.0375 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f22111721f0>
├── 'a' --> tensor([[ 0.2369, -0.9854,  0.8495],
│                   [-0.0787, -0.6408,  0.1677]])
└── 'x' --> <FastTreeValue 0x7f22111c9af0>
    └── 'c' --> tensor([[-1.8767,  0.5016, -0.4694,  0.3864],
                        [-0.7104, -0.9451,  0.5307,  0.4539],
                        [-0.1516, -0.4768, -0.9145,  0.4315]])

In [7]:
%timeit t.a = new_value

72 ns ± 0.031 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 1.0839, -0.7698, -0.9704],
               [ 0.4433, -0.7054,  1.1089]]),
    x: Batch(
           c: tensor([[-1.8767,  0.5016, -0.4694,  0.3864],
                      [-0.7104, -0.9451,  0.5307,  0.4539],
                      [-0.1516, -0.4768, -0.9145,  0.4315]]),
       ),
)

In [10]:
b.a

tensor([[ 1.0839, -0.7698, -0.9704],
        [ 0.4433, -0.7054,  1.1089]])

In [11]:
%timeit b.a

59 ns ± 0.0307 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 2.0479, -1.3631,  0.5041],
               [-0.0440,  0.7918,  1.2361]]),
    x: Batch(
           c: tensor([[-1.8767,  0.5016, -0.4694,  0.3864],
                      [-0.7104, -0.9451,  0.5307,  0.4539],
                      [-0.1516, -0.4768, -0.9145,  0.4315]]),
       ),
)

In [13]:
%timeit b.a = new_value

496 ns ± 0.115 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

848 ns ± 0.268 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

10.9 µs ± 18.4 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

142 µs ± 687 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

142 µs ± 661 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f21689fa280>
├── 'a' --> tensor([[[ 1.0839, -0.7698, -0.9704],
│                    [ 0.4433, -0.7054,  1.1089]],
│           
│                   [[ 1.0839, -0.7698, -0.9704],
│                    [ 0.4433, -0.7054,  1.1089]],
│           
│                   [[ 1.0839, -0.7698, -0.9704],
│                    [ 0.4433, -0.7054,  1.1089]],
│           
│                   [[ 1.0839, -0.7698, -0.9704],
│                    [ 0.4433, -0.7054,  1.1089]],
│           
│                   [[ 1.0839, -0.7698, -0.9704],
│                    [ 0.4433, -0.7054,  1.1089]],
│           
│                   [[ 1.0839, -0.7698, -0.9704],
│                    [ 0.4433, -0.7054,  1.1089]],
│           
│                   [[ 1.0839, -0.7698, -0.9704],
│                    [ 0.4433, -0.7054,  1.1089]],
│           
│                   [[ 1.0839, -0.7698, -0.9704],
│                    [ 0.4433, -0.7054,  1.1089]]])
└── 'x' --> <FastTreeValue 0x7f21689fab20>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.1 µs ± 31.3 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f2161d662b0>
├── 'a' --> tensor([[ 1.0839, -0.7698, -0.9704],
│                   [ 0.4433, -0.7054,  1.1089],
│                   [ 1.0839, -0.7698, -0.9704],
│                   [ 0.4433, -0.7054,  1.1089],
│                   [ 1.0839, -0.7698, -0.9704],
│                   [ 0.4433, -0.7054,  1.1089],
│                   [ 1.0839, -0.7698, -0.9704],
│                   [ 0.4433, -0.7054,  1.1089],
│                   [ 1.0839, -0.7698, -0.9704],
│                   [ 0.4433, -0.7054,  1.1089],
│                   [ 1.0839, -0.7698, -0.9704],
│                   [ 0.4433, -0.7054,  1.1089],
│                   [ 1.0839, -0.7698, -0.9704],
│                   [ 0.4433, -0.7054,  1.1089],
│                   [ 1.0839, -0.7698, -0.9704],
│                   [ 0.4433, -0.7054,  1.1089]])
└── 'x' --> <FastTreeValue 0x7f21689e8d30>
    └── 'c' --> tensor([[-1.8767,  0.5016, -0.4694,  0.3864],
                        [-0.7104, -0.9451,  0.5307,  0.4539],
                 

In [23]:
%timeit t_cat(trees)

30.2 µs ± 89.1 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

57.8 µs ± 72.3 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[-1.8767,  0.5016, -0.4694,  0.3864],
                       [-0.7104, -0.9451,  0.5307,  0.4539],
                       [-0.1516, -0.4768, -0.9145,  0.4315]],
              
                      [[-1.8767,  0.5016, -0.4694,  0.3864],
                       [-0.7104, -0.9451,  0.5307,  0.4539],
                       [-0.1516, -0.4768, -0.9145,  0.4315]],
              
                      [[-1.8767,  0.5016, -0.4694,  0.3864],
                       [-0.7104, -0.9451,  0.5307,  0.4539],
                       [-0.1516, -0.4768, -0.9145,  0.4315]],
              
                      [[-1.8767,  0.5016, -0.4694,  0.3864],
                       [-0.7104, -0.9451,  0.5307,  0.4539],
                       [-0.1516, -0.4768, -0.9145,  0.4315]],
              
                      [[-1.8767,  0.5016, -0.4694,  0.3864],
                       [-0.7104, -0.9451,  0.5307,  0.4539],
                       [-0.1516, -0.4768, -0.9145,  0.4315]],

In [26]:
%timeit Batch.stack(batches)

79 µs ± 278 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[-1.8767,  0.5016, -0.4694,  0.3864],
                      [-0.7104, -0.9451,  0.5307,  0.4539],
                      [-0.1516, -0.4768, -0.9145,  0.4315],
                      [-1.8767,  0.5016, -0.4694,  0.3864],
                      [-0.7104, -0.9451,  0.5307,  0.4539],
                      [-0.1516, -0.4768, -0.9145,  0.4315],
                      [-1.8767,  0.5016, -0.4694,  0.3864],
                      [-0.7104, -0.9451,  0.5307,  0.4539],
                      [-0.1516, -0.4768, -0.9145,  0.4315],
                      [-1.8767,  0.5016, -0.4694,  0.3864],
                      [-0.7104, -0.9451,  0.5307,  0.4539],
                      [-0.1516, -0.4768, -0.9145,  0.4315],
                      [-1.8767,  0.5016, -0.4694,  0.3864],
                      [-0.7104, -0.9451,  0.5307,  0.4539],
                      [-0.1516, -0.4768, -0.9145,  0.4315],
                      [-1.8767,  0.5016, -0.4694,  0.3864],
                   

In [28]:
%timeit Batch.cat(batches)

145 µs ± 1.16 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

320 µs ± 533 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
